# Fit Pixel

In this notebook we will fit a single pixel in a data cube for NGC628.

#### * If you have not yet downloaded the sample data cube, uncomment the below cell and run and download the sample data cube.  
This will save a 900MB file called `sample_data.hdf5` to the **`ExampleData`** directory in the **`LUCI`** folder. Do not be concerned if it takes a few minutes to download.

In [1]:
# !wget -O Data/NGC628_SN3.hdf5  https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/2307000z.hdf5?RUNID=xc9le6u8llecp7fp

In [2]:
# Imports
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

# Get location of LUCI 
path = os.path.abspath(os.path.pardir)  
sys.path.insert(0, path)  # add LUCI to the available paths

from LuciBase import Luci

%config Completer.use_jedi=False  # enable autocompletion when typing in Jupyter notebooks

2022-09-23 17:15:35.134280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /app/lib
2022-09-23 17:15:35.134321: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Set the required parameters. We will be using our machine learning algorithm to get the initial guesses - this happens under the hood in `LuciFit`, so that the user is not required to think about the initial guess

In [3]:
# Initialize paths and set parameters
Luci_path = path+'/'
cube_dir = '/mnt/carterrhea/carterrhea/NGC628'  # Full path to data cube
cube_name = 'NGC628_SN3.merged.cm1.1.0'  # don't add .hdf5 extension
object_name = 'NGC628'
redshift = 0.000133  # Redshift of object
resolution = 1000 # The actual resolution is 400, but we don't have ML algorithms for that resolution, so use 1000

Intialize our LUCI object

In [4]:
cube = Luci(Luci_path, cube_dir + '/' + cube_name, cube_dir, object_name, redshift, resolution)

Reading in data...


100%|██████████| 9/9 [00:12<00:00,  1.43s/it]


2022-09-23 17:15:50.892253: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /app/lib
2022-09-23 17:15:50.892282: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-23 17:15:50.892301: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-09-23 17:15:50.892484: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Create a deep frame

Let's extract a background region and take a look at it. The background region is defined in a ds9 region file in the `ExampleData` folder.

In [5]:
# We use 'mean = True' to take the mean of the emission in the region instead of the sum
bkg_axis, bkg_sky = cube.extract_spectrum_region(cube_dir+'/bkg.reg', mean=True)

100%|██████████| 2064/2064 [00:00<00:00, 2616.61it/s]


We will now fit a single pixel and take a look at the fit. This fit commands has all the same options as all the other commands except for binning :)

In [6]:
# Fit!
axis, sky, fit_dict = cube.fit_pixel(
    ['Halpha', 'NII6548', 'NII6583', 'SII6716', 'SII6731'],  # lines
    'sincgauss',   # fit function
    [1,1,1,1,1],  # velocity relationship 
    [1,1,1,1,1],  # sigma relationship
    1250, 1045,    # x & y coordinate
    bkg=bkg_sky, uncertainty_bool=True
)

IndexError: invalid index to scalar variable.

And now we can plot the results

In [ ]:
plt.plot(axis, sky, label='spectrum')
plt.plot(axis, fit_dict['fit_vector'], label='fit', linestyle='--')
plt.legend()
plt.xlim(15000, 16000)

And that is it! Congratulations, you have just used `LUCI`! 

In [ ]:
fit_dict['vels_errors']

In [ ]:
fit_dict